In [1]:
import pandas as pd
import numpy as np
import os.path # File I/O
import re # Useful for cleaning data
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import HashingVectorizer, TfidfVectorizer # I'll try both
from sklearn.svm import SVC # I think that using a SVM should be best for this problem
from sklearn.pipeline import make_pipeline
from textblob import TextBlob # To correct spelling errors throughout the dataframe.

# File Input

In [2]:
# Set up filepath to read in sentiment data
basepath = os.path.dirname('__file__')
filepath = os.path.abspath(os.path.join(basepath, 
                                       'Datasets',
                                       'ISEAR_0', 
                                       'isear_databank',
                                       'ISEAR_emotion.xlsx'))
print(filepath)

E:\_Sentiment\Datasets\ISEAR_0\isear_databank\ISEAR_emotion.xlsx


In [3]:
df = pd.read_excel(filepath)

In [4]:
df.head()

,ID,CITY,COUN,SUBJ,SEX,AGE,RELI,PRAC,FOCC,MOCC,...,SELF,RELA,VERBAL,NEUTRO,Field1,Field3,Field2,MYKEY,SIT,STATE
0,11001,1,1,1,1,33,1,2,6,1,...,3,3,2,0,joy,4,3,110011,"During the period of falling in love, each tim...",1
1,11001,1,1,1,1,33,1,2,6,1,...,2,2,0,0,fear,3,2,110012,When I was involved in a traffic accident.,1
2,11001,1,1,1,1,33,1,2,6,1,...,2,1,0,0,anger,1,3,110013,When I was driving home after several days of...,1
3,11001,1,1,1,1,33,1,2,6,1,...,1,1,0,2,sadness,4,4,110014,When I lost the person who meant the most to me.,1
4,11001,1,1,1,1,33,1,2,6,1,...,0,2,0,0,disgust,4,4,110015,The time I knocked a deer down - the sight of ...,1


Based on this, it looks like I'm after 'Field1' (emotion) and SIT (associated text)

Well, some users are problemmatic, so I need to include userID as well.

In [5]:
df = df[['ID','Field1', 'SIT']]
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7666 entries, 0 to 7665
Data columns (total 3 columns):
ID        7666 non-null int64
Field1    7666 non-null object
SIT       7666 non-null object
dtypes: int64(1), object(2)
memory usage: 179.8+ KB


In [6]:
line = df['SIT'][2]
line

'When I was driving home after  several days of hard work, there á\nwas a motorist ahead of me who was driving at 50 km/hour and á\nrefused, despite his low speeed to let me overtake.'

# Data Cleaning

In [7]:
# I need to figure out how to strip weird characters out of the text. This has effectively caught
# those pesky accented a's. Somehow they got bundled with newline characters. I'm curious why.
s = r'.\n'
re.sub(s, '', line)

'When I was driving home after  several days of hard work, there was a motorist ahead of me who was driving at 50 km/hour and refused, despite his low speeed to let me overtake.'

In [8]:
# This should do the trick.
def remove_accented_chars(line):
    pattern = r'.\n'
    return re.sub(pattern, '', line)

def remove_brackets(line):
    pattern1 = r'^\[ '
    pattern2 = r']$'
    s = re.sub(pattern1, '', line)
    return re.sub(pattern2, '', s)
    


In [9]:
df['SIT'] = df['SIT'].apply(remove_accented_chars)
df['SIT'] = df['SIT'].apply(remove_brackets)
df['SIT'][14]

'On days when I feel close to my partner and other friends.  When I feel at peace with myself and also experience a close contact with people whom I regard greatly.'

## N/A and duplicate answer cleaning

Looking at the data, special cases need to be handled / replaced. Some lines say things like... [Same as in anger] - which implies the response is the same as given for anger. Occasionally I've come across 'Nothing', 'Not applicable' or 'None' - these rows need to be deleted b/c they're of no use.

N/A answers: [ Do not know.] [ No response.] NO RESPONSE. Doesn't apply. Nothing. None. Not applicable. [ Normally I do not feel disgusted.] [ Can not think of anything just now.] [ Can not think of any situation.] [ Can not remember.] Can't think of any. [ Cannot recall any incident when I felt shame.] [ Do not remember any situation of that kind.] [ Can not think of anything.] [ I do not remember when I last felf ashamed. I do not usually feel ashamed of what I do.] Not applicable to myself. Can't remember any episode of disgust. Blank. Not included on questionnaire. Cannot recall the emotion with any force. Haven't been frightened for ages. Haven't felt shame for ages. Cannot remember such a situation. DO NOT REMEMBER. [ No description.] [ I have not felt this emotion.] [ Never felt the emotion.] [ I have not felt this emotion in my life.] NO RESPONSE (w/o the period) [ I have never felt this emotion.] [ I have felt shame but am unable to remember any particular incident.] [ I have not felt this emotion yet.] [ Never experienced.] [ Never] [ There are many instances which are all equally irratating.] [ Honestly, I have never felt disgust at any situation in my life.] [ Sorry, I was never ashamed about anything in my life.] [ I can positively say that I have never done anything that made me feel guilty.] [ I am quite shameless, not applicable.] [ Not applicable.] [ Do not remember any incident.] 

Duplicate answers: [Same as in anger.], [Same as above ... ], see answer for "shame"., The same as in "shame"., [ The same as in shame.'], [ The same as in anger.], The same event described under "shame"., As in sadness (A), relating to this slaghter of fur-seals., [ The same as in guilt.], The same as in SHAME.,

In [10]:
na_answers = '''Do not know.
No response.
NO RESPONSE.
Doesn't apply.
Nothing.
None.
Not applicable.
Normally I do not feel disgusted.
Can not think of anything just now.
Can not think of any situation.
Can not remember.
Can't think of anything.
I do not remember when I last felf ashamed. I do not usually feel ashamed of what I do.
Not applicable to myself.
Can't remember any episode of disgust.
Blank.
Not included on questionnaire.
Cannot recall the emotion with any force.
Haven't been frightened for ages.
Haven't felt shame for ages.
Cannot remember such a situation.
DO NOT REMEMBER.
No description.
I have not felt this emotion.
Never felt the emotion.
I have not felt this emotion in my life.
NO RESPONSE
I have never felt this emotion.
I have felt shame but am unable to remember any particular incident.
I have not felt this emotion yet.
Never experienced.
Never
There are many instances which are all equally irratating.
Honestly, I have never felt disgust at any situation in my life.
Sorry, I was never ashamed of anything in my life.
I can positively say that I have never done anything that made me feel guilty.
I am quite shameless, not applicable.
Do not remember any incident
Same as in anger.
see answer for "shame".
The same as in shame.
The same as in anger.
The same event described under "shame".
As in sadness (A), relating to this slaughter of fur seals.
The same as in guilt.
The same as in SHAME.'''
na_answers = na_answers.split('\n')

In [11]:
# Writing a function to replace all the non-ansers with np.nan so I can remove them effectively from the dataset.

def replace_non_answers(line):
    if line in na_answers:
        return np.nan
    return line

In [12]:
df['SIT'] = df['SIT'].apply(replace_non_answers)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7666 entries, 0 to 7665
Data columns (total 3 columns):
ID        7666 non-null int64
Field1    7666 non-null object
SIT       7530 non-null object
dtypes: int64(1), object(2)
memory usage: 179.8+ KB


## NOTE: 
User 261035 sucks dick. His answers aren't even worth catching. It's easier just to exclude him explicitly. Hey, look, he found a friend. 261058.

In [13]:
problem_users = [261035, 261058]

def remove_problem_users(row):
    if row['ID'] in problem_users:
        row['SIT'] = np.nan
        return row
    return row

In [14]:
# let's clip the dataset. It should be ready for processing at this point.
df = df.apply(remove_problem_users, axis=1).dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7516 entries, 0 to 7665
Data columns (total 3 columns):
ID        7516 non-null int64
Field1    7516 non-null object
SIT       7516 non-null object
dtypes: int64(1), object(2)
memory usage: 234.9+ KB


In [15]:
# The ID column isn't a useful feature - it was just helpful for cleaning. let's get rid of it.
df = df[['Field1', 'SIT']]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7516 entries, 0 to 7665
Data columns (total 2 columns):
Field1    7516 non-null object
SIT       7516 non-null object
dtypes: object(2)
memory usage: 176.2+ KB


After building / a model, it's become clear that there are still issues with the data in that a graph search yields a model with 56.7% accuracy. One of the things I neglected to account for was the ample spelling errors in the dataset. I found a stackoverflow post that might help me out here. https://stackoverflow.com/questions/35070452/how-to-correct-spelling-in-a-pandas-dataframe

In [16]:
df['SIT'] = df['SIT'].apply(lambda txt: ''.join(TextBlob(txt).correct()))

# Pipeline Building

Since there should be a rather large vocabulary, I think implementing a hashing vectorizer might be the correct choice. However, since there are drawbacks (no idf capabilities, no inverse transform capabilities...), I might as well give the tfidf vectorizer a shot first.

In [17]:
X, y = df['SIT'], df['Field1']
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=0)

In [18]:
pipeline = make_pipeline(TfidfVectorizer(), SVC())
param_grid = {'tfidfvectorizer__ngram_range': [(1, 1), (1, 2), (2, 2)], 
             'svc__C': [.01, .1, 1, 10, 100],
             'svc__gamma': [.01, .1, 1, 10]}
grid = GridSearchCV(pipeline, param_grid=param_grid, cv=10, n_jobs=-1, verbose = 2)
grid.fit(X_train, y_train)
print(grid.best_params_)
grid.score(X_test, y_test)

Fitting 10 folds for each of 60 candidates, totalling 600 fits


[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:  9.2min
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed: 22.5min
[Parallel(n_jobs=-1)]: Done 600 out of 600 | elapsed: 38.5min finished


{'tfidfvectorizer__ngram_range': (1, 2), 'svc__C': 10, 'svc__gamma': 1}


0.5779670037253859

In [19]:
# Curious to see if maybe we're over-fitting.
grid.score(X_train, y_train)

0.99911300337058717

It really looks like we might be. 

So after using TextBlob to correct spelling & playing with the random state param, it doesn't seem like for the time being that I'll get much more accurate than 55 - 60 %. I'm not sure if it's just the quality of the dataset that I'm using or the particular models I'm using -- everywhere I read seems to indicate that for a dataset of this size, SVM's are the way to go. I suppose SGD is an option, I've also seen Naive Bayes and Maximum Entropy classifiers used. NB would certainly be a simpler model.

Just noting, gridsearchcv found best params to be ngram_range = (1, 2), svc C = 10, svc gamma = 1. However, 57.7% test accuracy and 99.9% train accuracy may indicate overfitting.